In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras

sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

# Import and split the dataframe

In [ ]:
def per2float(x):
  try: 
    x = float(x.strip('%'))/100
    return x
  except: pass

train_df = pd.read_csv('../input/genshin-impact-character-base-stat/Keqing/KeqingStat.csv', converters={'CRIT DMG':per2float}, skipinitialspace=True)
test_df = pd.read_csv('../input/genshin-impact-character-base-stat/Keqing/KeqingStat.csv', converters={'CRIT DMG':per2float}, skipinitialspace=True)

train_df, test_df = train_df.drop([2, 4, 6, 8, 10, 12]), test_df.drop([2, 4, 6, 8, 10, 12]) #drop rows with "+" on levels
train_df['Lv'], test_df['Lv'] = train_df.Lv.astype(int), test_df.Lv.astype(int)

train_df.head()

# Visualise the data

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(20,8))

sns.scatterplot(x=train_df['Lv'], y=train_df['Base HP'], ax=ax1)
sns.scatterplot(x=train_df['Lv'], y=train_df['Base ATK'], ax=ax2)
sns.scatterplot(x=train_df['Lv'], y=train_df['Base DEF'], ax=ax3)
sns.scatterplot(x=train_df['Lv'], y=train_df['CRIT DMG'], ax=ax4)

plt.show()

# Create models for our prediction

In [ ]:
def HP_model():
  model = tf.keras.Sequential([keras.layers.Dense(1, input_shape=[1])],name='HP_Model',)
  model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(1), metrics=['accuracy'])
  return model

def ATK_model():
  model = tf.keras.Sequential([keras.layers.Dense(1, input_shape=[1])],name='ATK_Model')
  model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.05), metrics=['accuracy'])
  return model

def DEF_model():
  model = tf.keras.Sequential([keras.layers.Dense(1, input_shape=[1])],name='DEF_Model')
  model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.05), metrics=['accuracy'])
  return model

def CRIT_model():
  model = tf.keras.Sequential([keras.layers.Dense(1, input_shape=[1])],name='CRIT_Model')
  model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['accuracy'])
  return model

hp_m = HP_model()
atk_m = ATK_model()
def_m = DEF_model()
crit_m = CRIT_model()

# Train the model

In [ ]:
hp_his = hp_m.fit(train_df['Lv'], train_df['Base HP'], epochs=900, verbose=0)
atk_his = atk_m.fit(train_df['Lv'], train_df['Base ATK'], epochs=900, verbose=0)
def_his = def_m.fit(train_df['Lv'], train_df['Base DEF'], epochs=900, verbose=0)
crit_his = crit_m.fit(train_df['Lv'], train_df['CRIT DMG'], epochs=900, verbose=0)

# Visualise the training data

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(20,8))

ax1.plot(hp_his.history['loss'])
ax2.plot(atk_his.history['loss'])
ax3.plot(def_his.history['loss'])
ax4.plot(def_his.history['loss'])

ax1.set_xlabel('Epochs'), ax2.set_xlabel('Epochs'), ax3.set_xlabel('Epochs'), ax4.set_xlabel('Epochs'), ax4.set
ax1.set_ylabel('Loss'), ax2.set_ylabel('Loss'), ax3.set_ylabel('Loss'), ax4.set_ylabel('Loss')

plt.show()

# Predict the stats using our model we've trained

In [ ]:
result = {}

result['hp_prediction'] = hp_m.predict(test_df['Lv'])
result['atk_prediction'] = atk_m.predict(test_df['Lv'])
result['def_prediction'] = def_m.predict(test_df['Lv'])
result['crit_prediction'] = crit_m.predict(test_df['Lv'])

# Visualise the testing data & our prediction data

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(20,8))

sns.scatterplot(x=test_df['Lv'], y=test_df['Base HP'], ax=ax1)
ax1.plot(test_df['Lv'], result['hp_prediction'], color='g')

sns.scatterplot(x=test_df['Lv'], y=test_df['Base ATK'], ax=ax2)
ax2.plot(test_df['Lv'], result['atk_prediction'], color='r')

sns.scatterplot(x=test_df['Lv'], y=test_df['Base DEF'], ax=ax3)
ax3.plot(test_df['Lv'], result['def_prediction'], color='b')

sns.scatterplot(x=test_df['Lv'], y=test_df['CRIT DMG'], ax=ax4)
ax4.plot(test_df['Lv'], result['crit_prediction'], color='b')

plt.show()

# Output our predicted data as csv

In [ ]:
HP = np.hstack(result['hp_prediction']).astype(int)
ATK = np.hstack(result['atk_prediction']).astype(int)
DEF = np.hstack(result['def_prediction']).astype(int)
conv_crit = pd.DataFrame(result['crit_prediction']).applymap(lambda x: '{:.2%}'.format(x)).values
CRIT = np.hstack(conv_crit).astype(str)

data = {'Lv': test_df['Lv'], 'Base HP': HP, 'Base ATK': ATK, 'Base DEF': DEF, 'CRIT DMG': CRIT}

result_df = pd.DataFrame(data, columns= ['Lv', 'Base HP', 'Base ATK', 'Base DEF', 'CRIT DMG'])
result_df.to_csv('./prediction.csv')

result_df